In [ ]:
!pip install ultralytics
!pip install --quiet vietocr

In [ ]:
!python3 -m pip install paddlepaddle==3.1.0 -i https://www.paddlepaddle.org.cn/packages/stable/cpu/
!python3 -m pip install paddlepaddle-gpu==3.1.0 -i https://www.paddlepaddle.org.cn/packages/stable/cu126/
!pip install paddleocr

In [ ]:
# from ultralytics import YOLO
import os
import matplotlib.pyplot as plt
from PIL import Image
from paddleocr import TextDetection
import cv2
import numpy as np

In [ ]:
# Folder paths
input_folder = '/content/drive/MyDrive/sample_intern1/sample'
pdf_output_folder = '/content/drive/MyDrive/sample_intern1/output/pdf_first_pages'
img_output_folder = '/content/drive/MyDrive/sample_intern1/output/img_first_pages'
os.makedirs(pdf_output_folder, exist_ok=True)
os.makedirs(img_output_folder, exist_ok=True)

# Loop through PDF files
for filename in os.listdir(input_folder):
    if filename.lower().endswith('.pdf'):
        file_path = os.path.join(input_folder, filename)
        base_name = os.path.splitext(filename)[0]

        # Extract first page and save as new PDF
        reader = PdfReader(file_path)
        writer = PdfWriter()
        if len(reader.pages) > 0:
            writer.add_page(reader.pages[0])
            pdf_output_path = os.path.join(pdf_output_folder, f'{base_name}_page1.pdf')
            with open(pdf_output_path, 'wb') as out_pdf:
                writer.write(out_pdf)
            print(f'✅ Saved first page: {pdf_output_path}')

            # Convert first page to image
            images = convert_from_path(pdf_output_path, first_page=1, last_page=1)
            img_output_path = os.path.join(img_output_folder, f'{base_name}_page1.jpg')
            images[0].save(img_output_path, 'JPEG')
            print(f'🖼️ Converted to image: {img_output_path}')


In [ ]:
# Load a model
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data="/content/drive/MyDrive/sample_intern1/dataset.yaml", epochs=100, imgsz=640)

In [ ]:
# Load pretrained model
model = YOLO("/content/drive/MyDrive/sample_intern1/best.pt")  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data="/content/drive/MyDrive/sample_intern1/dataset.yaml", epochs=100, imgsz=640)

In [ ]:
# Đường dẫn đến thư mục chứa ảnh
image_folder = "/content/drive/MyDrive/sample_intern1/output/img_first_pages"
output_dir = "/content/drive/MyDrive/sample_intern1/output/test/"
os.makedirs(output_dir, exist_ok=True)

# Tải mô hình YOLO
model_crop = YOLO("/content/drive/MyDrive/sample_intern1/best.pt")

# Biến để đánh số thứ tự các ảnh crop
crop_count = 0

# Lặp qua tất cả các file ảnh trong thư mục
for filename in os.listdir(image_folder):
    # Kiểm tra xem file có phải là file ảnh không (jpg, jpeg, png, ...)
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        # Đường dẫn đầy đủ đến ảnh
        image_path = os.path.join(image_folder, filename)

        # Đọc ảnh
        image = cv2.imread(image_path)

        # Dự đoán trên ảnh
        results = model_crop(image)

        # Duyệt qua tất cả các kết quả dự đoán
        for result in results:
            # Duyệt qua từng box (bounding box) trong các kết quả dự đoán
            for box in result.boxes:
                # Lấy tọa độ của bounding box (x1, y1, x2, y2)
                x1, y1, x2, y2 = map(int, box.xyxy[0])

                # Trích xuất vùng ảnh trong bounding box
                object_image = image[y1:y2, x1:x2]

                # # Tạo tên file với số thứ tự
                output_filename = os.path.join(output_dir, f"crop_{crop_count}.jpg")

                # Lưu ảnh crop với tên file theo thứ tự
                cv2.imwrite(output_filename, object_image)

                # Tăng biến đếm
                crop_count += 1

                print(f"Đã lưu {output_filename}")


In [ ]:
from paddleocr import PaddleOCR
from vietocr.tool.predictor import Predictor
from vietocr.tool.config import Cfg
from PIL import Image, ImageDraw, ImageFont
import os
import numpy as np

# Khởi tạo PaddleOCR chỉ dùng text detection
ocr = PaddleOCR(use_angle_cls=False, lang='vi', text_detection_model_name="PP-OCRv5_mobile_det")

# Khởi tạo VietOCR cho text recognition
config = Cfg.load_config_from_name('vgg_transformer')
config['cnn']['pretrained'] = False
config['device'] = 'cuda:0'
vietocr_predictor = Predictor(config)

# Đường dẫn ảnh đầu vào
img_path = '/content/drive/MyDrive/sample_intern1/output/crops/crop_17.jpg'
image = Image.open(img_path).convert('RGB')
draw = ImageDraw.Draw(image)

# Font để vẽ text
try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 18)
except:
    font = ImageFont.load_default()

recognized_text_list = []
# Tạo thư mục output nếu chưa có
os.makedirs("output/crops", exist_ok=True)

# Text detection bằng PaddleOCR
result = ocr.ocr(img_path)

# Lấy dt_polys từ kết quả
if result and isinstance(result[0], dict) and 'dt_polys' in result[0]:
    for idx, polygon in enumerate(result[0]['dt_polys']):
        # polygon là numpy array (4, 2), convert thành list point tuple
        box_coords = [tuple(map(int, pt)) for pt in polygon]

        # Tìm bounding box
        xs = [pt[0] for pt in box_coords]
        ys = [pt[1] for pt in box_coords]
        x_min, x_max = min(xs), max(xs)
        y_min, y_max = min(ys), max(ys)

        # Crop vùng ảnh
        cropped = image.crop((x_min, y_min, x_max, y_max))

        # Lưu ảnh crop
        crop_path = f"output/crops/crop_{idx}.jpg"
        cropped.save(crop_path)

        # Nhận dạng văn bản bằng VietOCR
        try:
            recognized_text = vietocr_predictor.predict(cropped)
            recognized_text_list.append(recognized_text)
        except:
            recognized_text = "[ERROR]"

        # Vẽ polygon + text lên ảnh gốc
        draw.line(box_coords + [box_coords[0]], fill=(255, 0, 0), width=2)
        draw.text((x_min, y_min - 10), recognized_text, fill=(0, 0, 255), font=font)

# Lưu ảnh kết quả
image.save("output/result_vietocr.jpg")


In [ ]:
recognized_text_list